# MeerKAT Data

MeerKAT observations have different sizes from some GB to TB. To provide access to such data sets, [katdal](https://github.com/ska-sa/katdal) provides an interface. The data products are available as *MeerKAT Visibility Format (MVF)*. [katdal](https://github.com/ska-sa/katdal) additionally offers a script which converts the *MVF* to CASA measurement *.ms* files.

In [ ]:
#!pip install katdal
import katdal
import time, yaml

The observation we used to run this Notebook is:

- Observer: Operator
- Experiment ID: 20210605-0009
- Description: 'Meertime phase up with flatten bandpass'
- Observed from 2021-06-06 01:06:13.567 CEST to 2021-06-06 01:13:09.646 CEST
- Dump rate / period: 0.12498 Hz / 8.002 s

If you want to run the same configurations, please insert the according token in `config.yaml`.

## Access to Data

MeerKAT observation files are available through the [MeerKAT archive](https://archive.sarao.ac.za/). It requires a registration and login to get access. A detailed description is accessible through [Archive Interface User Guide](https://archive.sarao.ac.za/statics/Archive_Interface_User_Guide.pdf).

To get access through [katdal](https://github.com/ska-sa/katdal) you have to generate an *rdp-link with token* on the [MeerKAT archive](https://archive.sarao.ac.za/) and copy the token into the `katdal.open` token argument. The URL doesn't need an update since it should never change. 

**Note: all tokens have expiry dates**

In [ ]:
with open('../config.yaml', 'r') as stream:
    token = yaml.safe_load(stream)['MeerKAT_archive_token']

stime = time.time()
data = katdal.open('https://archive-gw-1.kat.ac.za/1622934371/1622934371_sdp_l0.full.rdb',
                   s3_endpoint_url='https://archive-gw-1.kat.ac.za',
                   token=token)
print('time to read file = {} s'.format(time.time() - stime))
print('(dumps x channels x baselines) = {}'.format(data.shape))
print(data.vis.dataset)

## Observation Details

In [ ]:
print(data)

data.shape\[0\]: dumps

data.shape\[1\]: channels

data.shape\[2\]: correlation products

In [ ]:
data.shape

esimated memory of visibilities in Bytes: dumps \* channels \* correlation_products \* 64(complex64) / 8

In [ ]:
vis_memory = int(data.shape[0]*data.shape[1]*data.shape[2]*64/8)
vis_memory

## Using the Data

[katdal](https://github.com/ska-sa/katdal) provides functionality regarding filtering through `data.select`. This pre-filters the data so that when `data.vis` is accessed, not all the data has to be loaded. After that an `npumpy.ndarray` is returned, on which you can continue working as usual.

The following chunks show a small example of data access. Many parameter settings can be derived from the output of "print(data)".

select `timerange` from **Scans**

select `targets` from **Targets**

In [ ]:
data.select(timerange=('2021-06-05 23:11:53', '2021-06-05 23:13:05'), targets=[0])

select `spw` from **Spectral Windows**

select `subarray` from **Subarrays**

In [ ]:
data.select(spw=0, subarray=0)

select `ants` from **ants** or **Antennas**

select `pol` (polarisation) from {'H', 'V', 'HH', 'VV', 'HV', 'VH'}

select `scans` from **Scans ScanState**

select `freqrange` NOT sure yet

In [ ]:
data.select(ants='m000,m001,m002,m003,m005,m006,m007,m008,m009',
            pol='H', scans=(0,1,2), freqrange=(1700e6, 1800e6))

[katdal](https://github.com/ska-sa/katdal) provides more filter functionality. The code can be found [here](https://github.com/ska-sa/katdal/blob/master/katdal/dataset.py) in `DataSet`.

Note the `[:]` indexing, as the `vis` and `timestamps` properties are special `LazyIndexer` objects that only give you the actual data when you use indexing, in order not to inadvertently load the entire array into memory.

For the example dataset and no selection the `vis` array will have a shape of `data.shape`. The time dimension is labelled by `d.timestamps`, the frequency dimension by `d.channel_freqs` and the correlation product dimension by `d.corr_products`.

In [ ]:
#vis = data.vis[:] # loads the actual visibilities to `vis` as numpy.ndarray

For a detailed description of how to use the library you can visit [katdal](https://github.com/ska-sa/katdal) repositroy. In addition, [MeerKAT-Cookbook](https://github.com/ska-sa/MeerKAT-Cookbook) also provides some examples of different use-cases using [katdal](https://github.com/ska-sa/katdal).